In [1]:
from scripts_stock.cfg.set_dir import *
import pandas as pd

project_dir = ProjectDir()

In [5]:
# from scripts_stock.cfg.owner_url_list_sina import DataURLSina

# data_url_sina = DataURLSina()
# data_url_sina.owner_shebao406()

('sina_owner_shebao406',
 'https://vip.stock.finance.sina.com.cn/corp/view/vCI_HoldStockState.php?stockid=600867&stockholderid=70010406')

In [2]:
df1 = pd.read_csv(os.path.join(project_dir.download_data_dir,"owner_sina","sina_owner_abudabi.csv"),encoding="utf-8-sig")
print(df1.head(5))

  stock_name  hold_num  hold_ratio stock_type update_date          owner_name
0       通化东宝  36391629        1.83      流通A股,  2024-03-31  sina_owner_abudabi
1       通化东宝  31836912        1.61      流通A股,  2024-07-23  sina_owner_abudabi
2       通化东宝  31870412        1.61      流通A股,  2024-07-10  sina_owner_abudabi
3       通化东宝  31914212        1.61      流通A股,  2024-07-08  sina_owner_abudabi
4       通化东宝  31914812        1.61      流通A股,  2024-06-30  sina_owner_abudabi


In [57]:


def get_increase_data(df1,quater_end1,quater_end2):
    df1 = df1[(df1["update_date"]==quater_end1) | (df1["update_date"]==quater_end2)].drop_duplicates()
    #print(df1)

    # 统计 'value' 列中每个唯一值的数量
    value_counts = df1['stock_name'].value_counts()
    # 筛选出计数大于等于3的唯一值
    unique_values_with_more_than_two = value_counts[value_counts >= 2].index.tolist()
    # 使用这些唯一值来过滤原始 DataFrame
    filtered_df = df1[df1['stock_name'].isin(unique_values_with_more_than_two)]

    last_two_dates = filtered_df.groupby('stock_name')['update_date'].apply(lambda x: x.sort_values().tail(3)).reset_index(name='update_date')
    #print(last_two_dates[last_two_dates["stock_name"]=="通化东宝"])

    merged_df = df1.merge(last_two_dates,on=['stock_name', 'update_date'], how='inner')
    #print(merged_df)
    result = merged_df.pivot(index='stock_name', columns='update_date', values='hold_num').reset_index()
    result["diff"] =  result[quater_end2]- result[quater_end1]
    result_increase = result[result["diff"]>0]
    result_increase["owner_name"] = df1["owner_name"].values[0]
    #print(result_increase[["stock_name",quater_end1,quater_end2,"diff"]].sort_values("diff",ascending=False))

    result_decrease = result[result["diff"]<0]
    result_decrease["owner_name"] = df1["owner_name"].values[0]
    #print(result_decrease[["stock_name",quater_end1,quater_end2,"diff"]].sort_values("diff"))
    return result_increase,result_decrease

# result_increase,result_decrease = get_increase_data(df1,quater_end1,quater_end2)

In [75]:
from scripts_stock.utils.common import CommonScript
conn = CommonScript.connect_to_db("test.db")
input_sql_str = "select * from prd_t_owner_sina"
df1 = pd.read_sql_query(input_sql_str, conn)


In [81]:

quater_end1 = "2024-03-31"
quater_end2 = "2024-06-30"
df_list = []
for index,group in df1.groupby("owner_name"):
    try:
        a1_in,a1_de = get_increase_data(group,quater_end1,quater_end2)
        df_list.append(a1_in)
    except:
        print(group["owner_name"].values[0])

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


sina_owner_CITIGROUP


In [77]:
df_list

[update_date stock_name  2024-03-31  2024-06-30     diff  \
 0                 坤彩科技     7018027     9009694  1991667   
 
 update_date               owner_name  
 0            sina_owne_merrill_lynch  ,
 update_date stock_name  2024-03-31  2024-06-30      diff           owner_name
 0                *ST亚星     1379758     1890387    510629  sina_owner_JPMorgan
 1                  三柏硕      339332      563632    224300  sina_owner_JPMorgan
 4                 中铁装配     4659555     4914285    254730  sina_owner_JPMorgan
 5                 分众传媒   159442204   177566666  18124462  sina_owner_JPMorgan
 7                 广哈通信      365764      678349    312585  sina_owner_JPMorgan
 9                 文一科技      405801      538751    132950  sina_owner_JPMorgan
 13                 达意隆     1184701     2339601   1154900  sina_owner_JPMorgan
 14                金浦钛业     2901739     4488914   1587175  sina_owner_JPMorgan
 15                铭普光磁     1144856     1169790     24934  sina_owner_JPMorgan
 16    

In [70]:
df1 = group
df1 = df1[(df1["update_date"]==quater_end1) | (df1["update_date"]==quater_end2)].drop_duplicates()
# 统计 'value' 列中每个唯一值的数量
value_counts = df1['stock_name'].value_counts()
# 筛选出计数大于等于3的唯一值
unique_values_with_more_than_two = value_counts[value_counts >= 2].index.tolist()
# 使用这些唯一值来过滤原始 DataFrame
filtered_df = df1[df1['stock_name'].isin(unique_values_with_more_than_two)]

last_two_dates = filtered_df.groupby('stock_name')['update_date'].apply(lambda x: x.sort_values().tail(3)).reset_index(name='update_date')
#print(last_two_dates[last_two_dates["stock_name"]=="通化东宝"])

#merged_df = df1.merge(last_two_dates,on=['stock_name', 'update_date'], how='inner')

In [78]:
df2 = pd.concat(df_list).reset_index()
df2[df2.columns[1:]]

update_date,stock_name,2024-03-31,2024-06-30,diff,owner_name
0,坤彩科技,7018027,9009694,1991667,sina_owne_merrill_lynch
1,*ST亚星,1379758,1890387,510629,sina_owner_JPMorgan
2,三柏硕,339332,563632,224300,sina_owner_JPMorgan
3,中铁装配,4659555,4914285,254730,sina_owner_JPMorgan
4,分众传媒,159442204,177566666,18124462,sina_owner_JPMorgan
...,...,...,...,...,...
900,丽珠B,4955526,5520426,564900,sina_owner_maucau_finance
901,丽珠集团,4955526,5520426,564900,sina_owner_maucau_finance
902,岱美股份,18030446,23439580,5409134,sina_owner_maucau_finance
903,金诚信,3944128,6016528,2072400,sina_owner_maucau_finance


In [49]:
from scripts_stock.utils.common import CommonScript
conn = CommonScript.connect_to_db("test.db")

input_sql_str = """
select t1.*,t2.stock_index,t3.avg_close,t3.avg_close*diff as incease_cash
from t_sina_owner_increase t1
join prd_t_all_stock_index_name t2
on t1.stock_name=t2.stock_name
left join 
(
  select stock_index,avg(stock_close_sep) as avg_close
  from prd_t_all_stock_close_sep
  group by stock_index
) t3
on t2.stock_index=t3.stock_index
"""
df1 = pd.read_sql_query(input_sql_str, conn)


In [52]:
df2 = df1.dropna().sort_values("diff",ascending=False)

#df2.to_csv(os.path.join(project_dir.analysis_data_dir,"owner_sina_increase_out.csv"),index=0,encoding="utf-8-sig")
df2.tail(20)

,stock_name,2024-03-31,2024-06-30,diff,owner_name,update_time,stock_index,avg_close,incease_cash
663,羚锐制药,10005002,10053283,48281,sina_owner_hkjiesuan,2024-09-29 22:48:28.243414,600285,0.00,0.00
29,三美股份,8258170,8304538,46368,sina_owner_hkjiesuan,2024-09-29 22:48:28.243414,603379,27.21,1261673.28
211,北方稀土,100483741,100529383,45642,sina_owner_hkjiesuan,2024-09-29 22:48:28.243414,600111,0.00,0.00
56,中交地产,2332451,2378078,45627,sina_owner_hkjiesuan,2024-09-29 22:48:28.243414,736,0.00,0.00
305,夏厦精密,82561,126009,43448,sina_owner_hkjiesuan,2024-09-29 22:48:28.243414,1306,40.52,1760512.96
332,奇正藏药,1592266,1633848,41582,sina_owner_hkjiesuan,2024-09-29 22:48:28.243414,2287,0.00,0.00
555,海汽集团,827176,864445,37269,sina_owner_hkjiesuan,2024-09-29 22:48:28.243414,603069,0.00,0.00
508,步科股份,367365,404175,36810,sina_owner_hkjiesuan,2024-09-29 22:48:28.243414,688160,0.00,0.00
741,金卡智能,5484156,5520332,36176,sina_owner_hkjiesuan,2024-09-29 22:48:28.243414,300349,0.00,0.00
221,华峰铝业,7350722,7385578,34856,sina_owner_hkjiesuan,2024-09-29 22:48:28.243414,601702,0.00,0.00


In [7]:
df1 = pd.read_csv(os.path.join(project_dir.download_data_dir,"owner_sina","sina_owner_abudabi.csv"),encoding="utf-8-sig")
df1 = pd.read_csv(os.path.join(project_dir.download_data_dir,"owner_sina","sina_owner_hkjiesuan.csv"),encoding="utf-8-sig")

quater_end1 = "2024-03-31"
quater_end2 = "2024-06-30"
result_increase["owner_name"] = df1["owner_name"].values[0]

In [8]:
result_increase

update_date,stock_name,2024-03-31,2024-06-30,diff,owner_name
1,*ST合泰,25239748,31663502,6423754,sina_owner_hkjiesuan
6,ST中南,21944855,28291883,6347028,sina_owner_hkjiesuan
7,ST亿利,19430846,22552978,3122132,sina_owner_hkjiesuan
13,ST旭蓝,8394302,10500461,2106159,sina_owner_hkjiesuan
15,ST爱康,49005764,70180104,21174340,sina_owner_hkjiesuan
...,...,...,...,...,...
1797,黑猫股份,6476057,10637383,4161326,sina_owner_hkjiesuan
1798,鼎捷软件,1974214,2809106,834892,sina_owner_hkjiesuan
1801,龙佰集团,63383581,70054694,6671113,sina_owner_hkjiesuan
1803,龙腾光电,5122569,7160894,2038325,sina_owner_hkjiesuan


In [15]:
result_increase.sort_values("diff",ascending=False)

update_date,stock_name,2024-03-31,2024-06-30,diff
330,京东方A,2624659006,3015216738,390557732
331,京东方B,2624659006,3015216738,390557732
1686,铜陵有色,295455387,649106156,353650769
536,南山铝业,707485039,1036918234,329433195
1209,浦发银行,701560730,931995460,230434730
...,...,...,...,...
246,中科星图,3904537,3917396,12859
216,中富电路,471087,483827,12740
1258,深南电路,9678952,9689373,10421
1382,省广集团,8920284,8923571,3287


In [1]:

project_dir = ProjectDir()
df1 = pd.read_csv(os.path.join(project_dir.download_data_dir,"owner_sina","sina_owner_abudabi.csv"),encoding="utf-8-sig")
df2 = df1.groupby("stock_name").agg({'hold_num': [ 'min', 'max']})
print(df2.reset_index())


    stock_name  hold_num          
                     min       max
0        TCL中环  22013860  23082610
1     •NÐË¹É·Ý   2643600   2643600
2         万华化学  19347676  22089357
3         万孚生物   1778646   4749947
4         万马股份   3078400   3078400
..         ...       ...       ...
199       驰宏锌锗   4866616   4866616
200       鸿路钢构   3894291   4047691
201       鼎汉技术   4268225   4268225
202       龙净环保   2083103   2083103
203       龙源技术   2762605   2762605

[204 rows x 3 columns]


In [7]:
import os 
import pandas as pd
from scripts_stock.cfg.set_dir import ProjectDir
a1 = os.path.join(ProjectDir.data_dir,"raw_data/all_stock_qianlong_20240929.csv")
df1 = pd.read_csv(a1,encoding="gbk")
print(df1)

     stock_name  stock_index
0          浦发银行       600000
1          白云机场       600004
2          东风股份       600006
3          中国国贸       600007
4          首创环保       600008
...         ...          ...
5346       美心翼申       873833
5347       万达轴承       920002
5348       成电光信       920008
5349       中草香料       920016
5350       太湖远大       920118

[5351 rows x 2 columns]
